In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

In [2]:
from DiversityEnsembleClassifier import DiversityEnsembleClassifier

### Loading datasets

In [3]:
breast = datasets.load_breast_cancer()
iris   = datasets.load_iris()
wine   = datasets.load_wine()
yeast  = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data', header=None, delim_whitespace=True)
spam   = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data', header=None)

<h3>Testing method</h3>

In [4]:
def compare_results(data, target):
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

    n_samples = ((X_train.shape[0] * 4) // 5)-1
    alg = {
                KNeighborsClassifier: {'n_neighbors':[1, n_samples]},
                SVC: {'C':[1, 1000],
                      'gamma':[0.001, 0.0001]
                      },
                DecisionTreeClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples]},
                RandomForestClassifier: {'min_samples_leaf':[1, n_samples], 'max_depth':[1, n_samples],
                                         'n_estimators':[100, 100]},
                GaussianNB: {},
                LinearDiscriminantAnalysis: {}
          }
    dec = DiversityEnsembleClassifier(alg, population_size=25, max_epochs=100, random_state=42)
    dec.fit(X_train,  y_train)
    dec_predict = dec.predict(X_test)

    rf = RandomForestClassifier(n_estimators=1000)
    rf.fit(X_train, y_train)
    rf_predict = rf.predict(X_test)

    ada = AdaBoostClassifier(n_estimators=1000)
    ada.fit(X_train, y_train)
    ada_predict = ada.predict(X_test)
    print()
    print('#'*60)
    print('Results')
    print('#'*60)
    print()
    
    print('-'*60)
    print('Diversity-based Ensemble Classifier')
    print('-'*60)
    print('Accuracy :', accuracy_score(y_test, dec_predict))
    try: print('F1-score :', f1_score(y_test, dec_predict))
    except: pass
    try: print('Precision:', precision_score(y_test, dec_predict))
    except: pass
    try: print('Recall   :', recall_score(y_test, dec_predict))
    except: pass
    try: print('ROC AUC  :', roc_auc_score(y_test, dec_predict))
    except: pass

    print('-'*60)
    print('Random Forest Classifier')
    print('-'*60)
    print('Accuracy :', accuracy_score(y_test, rf_predict))
    try: print('F1-score :', f1_score(y_test, rf_predict))
    except: pass
    try: print('Precision:', precision_score(y_test, rf_predict))
    except: pass
    try: print('Recall   :', recall_score(y_test, rf_predict))
    except: pass
    try: print('ROC AUC  :', roc_auc_score(y_test, rf_predict))
    except: pass

    print('-'*60)
    print('Ada Boost Classifer')
    print('-'*60)
    print('Accuracy :', accuracy_score(y_test, ada_predict))
    try: print('F1-score :', f1_score(y_test, ada_predict))
    except: pass
    try: print('Precision:', precision_score(y_test, ada_predict))
    except: pass
    try: print('Recall   :', recall_score(y_test, ada_predict))
    except: pass
    try: print('ROC AUC  :', roc_auc_score(y_test, ada_predict))
    except: pass


In [5]:
compare_results(data=breast.data, target=breast.target)

Starting genetic algorithm...
------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 3952 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.18849450549450547
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 54 ms
Fitting and predicting population...done in 6404 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.19026739926739925
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 59 ms
Fitting and predicting population...done in 6617 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.1938168498168498
---------------------

Fitting and predicting population...done in 7355 ms
Applying diversity selection...done in 6 ms
New population diversity measure: 0.2479157509157509
------------------------------------------------------------
Epoch 27
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 6998 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.2479157509157509
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...done in 7267 ms
Applying diversity selection...done in 6 ms
New population diversity measure: 0.2479157509157509
------------------------------------------------------------
Epoch 29
------------------------------------------------------------
Generating offspring...done in 59 ms
Fitting and predicting population...done in 6921 

Fitting and predicting population...done in 7081 ms
Applying diversity selection...done in 5 ms
New population diversity measure: 0.2517838827838828
------------------------------------------------------------
Epoch 53
------------------------------------------------------------
Generating offspring...done in 62 ms
Fitting and predicting population...done in 6976 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.2517838827838828
------------------------------------------------------------
Epoch 54
------------------------------------------------------------
Generating offspring...done in 62 ms
Fitting and predicting population...done in 7105 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.2517838827838828
------------------------------------------------------------
Epoch 55
------------------------------------------------------------
Generating offspring...done in 61 ms
Fitting and predicting population...done in 7094 

Fitting and predicting population...done in 7038 ms
Applying diversity selection...done in 4 ms
New population diversity measure: 0.2521611721611722
------------------------------------------------------------
Epoch 79
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...done in 7001 ms
Applying diversity selection...done in 5 ms
New population diversity measure: 0.2521611721611722
------------------------------------------------------------
Epoch 80
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...done in 7010 ms
Applying diversity selection...done in 5 ms
New population diversity measure: 0.2521611721611722
------------------------------------------------------------
Epoch 81
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 7061 

In [6]:
compare_results(data=iris.data, target=iris.target)

Starting genetic algorithm...
------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 5 ms
Fitting and predicting population...done in 3060 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.3878055555555555
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 61 ms
Fitting and predicting population...done in 5698 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.3930833333333334
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 87 ms
Fitting and predicting population...done in 8497 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.40022222222222226
----------------------

Fitting and predicting population...done in 5748 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.4281388888888889
------------------------------------------------------------
Epoch 27
------------------------------------------------------------
Generating offspring...done in 63 ms
Fitting and predicting population...done in 5806 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.4281388888888889
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 65 ms
Fitting and predicting population...done in 5721 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.4283472222222222
------------------------------------------------------------
Epoch 29
------------------------------------------------------------
Generating offspring...done in 69 ms
Fitting and predicting population...done in 6887 

Fitting and predicting population...done in 5610 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.4287916666666666
------------------------------------------------------------
Epoch 53
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...done in 5529 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.4287916666666666
------------------------------------------------------------
Epoch 54
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 5518 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.4287916666666666
------------------------------------------------------------
Epoch 55
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...done in 5537 

Fitting and predicting population...done in 5810 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.42895833333333333
------------------------------------------------------------
Epoch 79
------------------------------------------------------------
Generating offspring...done in 59 ms
Fitting and predicting population...done in 5836 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.42895833333333333
------------------------------------------------------------
Epoch 80
------------------------------------------------------------
Generating offspring...done in 62 ms
Fitting and predicting population...done in 5856 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.42895833333333333
------------------------------------------------------------
Epoch 81
------------------------------------------------------------
Generating offspring...done in 59 ms
Fitting and predicting population...done in 56

In [7]:
compare_results(data=wine.data, target=wine.target)

Starting genetic algorithm...
------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 5 ms
Fitting and predicting population...done in 2984 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.34370892018779337
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 59 ms
Fitting and predicting population...done in 5656 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.3517723004694836
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 70 ms
Fitting and predicting population...done in 6734 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.35386150234741787
---------------------

Fitting and predicting population...done in 3594 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.38982394366197176
------------------------------------------------------------
Epoch 27
------------------------------------------------------------
Generating offspring...done in 39 ms
Fitting and predicting population...done in 3633 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.38982394366197176
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 40 ms
Fitting and predicting population...done in 3624 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.38982394366197176
------------------------------------------------------------
Epoch 29
------------------------------------------------------------
Generating offspring...done in 45 ms
Fitting and predicting population...done in 36

Fitting and predicting population...done in 3737 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.39368544600938976
------------------------------------------------------------
Epoch 53
------------------------------------------------------------
Generating offspring...done in 45 ms
Fitting and predicting population...done in 3650 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.39368544600938976
------------------------------------------------------------
Epoch 54
------------------------------------------------------------
Generating offspring...done in 44 ms
Fitting and predicting population...done in 3662 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.39368544600938976
------------------------------------------------------------
Epoch 55
------------------------------------------------------------
Generating offspring...done in 44 ms
Fitting and predicting population...done in 36

Fitting and predicting population...done in 2478 ms
Applying diversity selection...done in 2 ms
New population diversity measure: 0.3944953051643192
------------------------------------------------------------
Epoch 79
------------------------------------------------------------
Generating offspring...done in 30 ms
Fitting and predicting population...done in 2500 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.3944953051643192
------------------------------------------------------------
Epoch 80
------------------------------------------------------------
Generating offspring...done in 29 ms
Fitting and predicting population...done in 2442 ms
Applying diversity selection...done in 3 ms
New population diversity measure: 0.3944953051643192
------------------------------------------------------------
Epoch 81
------------------------------------------------------------
Generating offspring...done in 31 ms
Fitting and predicting population...done in 2435 

In [11]:
yeast.iloc[:, -1].values

array(['MIT', 'MIT', 'MIT', ..., 'ME2', 'NUC', 'CYT'], dtype=object)

In [8]:
compare_results(data=yeast.iloc[:, 1:-1].values, target=yeast.iloc[:, -1].values)

Starting genetic algorithm...
------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 5 ms
Fitting and predicting population...

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


done in 7873 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.3380160067396798
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 60 ms
Fitting and predicting population...

C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\Alan\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


done in 11863 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.34412664981746693
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 71 ms
Fitting and predicting population...done in 14070 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.3465150238696995
------------------------------------------------------------
Epoch 3
------------------------------------------------------------
Generating offspring...done in 90 ms
Fitting and predicting population...done in 13457 ms
Applying diversity selection...done in 14 ms
New population diversity measure: 0.3471946082561078
------------------------------------------------------------
Epoch 4
------------------------------------------------------------
Generating offspring...done in 79 ms
Fitting and predicting population...done in 13339 ms
Applying diversity selection

Fitting and predicting population...done in 2288 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.3716456051670879
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 10 ms
Fitting and predicting population...done in 2125 ms
Applying diversity selection...done in 14 ms
New population diversity measure: 0.3716456051670879
------------------------------------------------------------
Epoch 29
------------------------------------------------------------
Generating offspring...done in 7 ms
Fitting and predicting population...done in 2226 ms
Applying diversity selection...done in 14 ms
New population diversity measure: 0.3716456051670879
------------------------------------------------------------
Epoch 30
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 1922

Fitting and predicting population...done in 1922 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.37257933164841334
------------------------------------------------------------
Epoch 54
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 1844 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.37257933164841334
------------------------------------------------------------
Epoch 55
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 1903 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.37257933164841334
------------------------------------------------------------
Epoch 56
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 20

Fitting and predicting population...done in 1781 ms
Applying diversity selection...done in 15 ms
New population diversity measure: 0.37315782083684357
------------------------------------------------------------
Epoch 80
------------------------------------------------------------
Generating offspring...done in 8 ms
Fitting and predicting population...done in 1963 ms
Applying diversity selection...done in 14 ms
New population diversity measure: 0.3733277169334457
------------------------------------------------------------
Epoch 81
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 1934 ms
Applying diversity selection...done in 14 ms
New population diversity measure: 0.37343162033136756
------------------------------------------------------------
Epoch 82
------------------------------------------------------------
Generating offspring...done in 9 ms
Fitting and predicting population...done in 193

ValueError: could not convert string to float: 'VAC'

In [9]:
compare_results(data=spam.iloc[:, :-1].values, target=spam.iloc[:, -1].values)

Starting genetic algorithm...
------------------------------------------------------------
Epoch 0
------------------------------------------------------------
Generating offspring...done in 4 ms
Fitting and predicting population...done in 75864 ms
Applying diversity selection...done in 33 ms
New population diversity measure: 0.29190806159420285
------------------------------------------------------------
Epoch 1
------------------------------------------------------------
Generating offspring...done in 61 ms
Fitting and predicting population...done in 44140 ms
Applying diversity selection...done in 38 ms
New population diversity measure: 0.29419972826086954
------------------------------------------------------------
Epoch 2
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 41755 ms
Applying diversity selection...done in 36 ms
New population diversity measure: 0.2974809782608696
---------------

Fitting and predicting population...done in 27882 ms
Applying diversity selection...done in 39 ms
New population diversity measure: 0.31703396739130435
------------------------------------------------------------
Epoch 26
------------------------------------------------------------
Generating offspring...done in 57 ms
Fitting and predicting population...done in 26768 ms
Applying diversity selection...done in 40 ms
New population diversity measure: 0.31703396739130435
------------------------------------------------------------
Epoch 27
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 26057 ms
Applying diversity selection...done in 37 ms
New population diversity measure: 0.31741530797101447
------------------------------------------------------------
Epoch 28
------------------------------------------------------------
Generating offspring...done in 57 ms
Fitting and predicting population...done

Fitting and predicting population...done in 25847 ms
Applying diversity selection...done in 40 ms
New population diversity measure: 0.3186553442028986
------------------------------------------------------------
Epoch 52
------------------------------------------------------------
Generating offspring...done in 57 ms
Fitting and predicting population...done in 26216 ms
Applying diversity selection...done in 37 ms
New population diversity measure: 0.3186553442028986
------------------------------------------------------------
Epoch 53
------------------------------------------------------------
Generating offspring...done in 57 ms
Fitting and predicting population...done in 25760 ms
Applying diversity selection...done in 38 ms
New population diversity measure: 0.3186553442028986
------------------------------------------------------------
Epoch 54
------------------------------------------------------------
Generating offspring...done in 56 ms
Fitting and predicting population...done in

Fitting and predicting population...done in 24674 ms
Applying diversity selection...done in 37 ms
New population diversity measure: 0.3242921195652175
------------------------------------------------------------
Epoch 78
------------------------------------------------------------
Generating offspring...done in 58 ms
Fitting and predicting population...done in 24923 ms
Applying diversity selection...done in 41 ms
New population diversity measure: 0.3257255434782608
------------------------------------------------------------
Epoch 79
------------------------------------------------------------
Generating offspring...done in 67 ms
Fitting and predicting population...done in 13853 ms
Applying diversity selection...done in 36 ms
New population diversity measure: 0.3257255434782608
------------------------------------------------------------
Epoch 80
------------------------------------------------------------
Generating offspring...done in 69 ms
Fitting and predicting population...done in